In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **HW 2: Classification**
In *homework 2*, you need to finish:

1.  Basic Part:

> *   Step 1: Load the input Data
> *   Step 2: Implement Naive Bayesian classifier
> *   Step 3: Build the classifier and check the correctness of Table building
> *   Step 4: Split Data
> *   Step 5: Make prediction and perform evaluation
> *   Step 6: Generate results

2.  Advanced Part:

> *   Step 1: Load the input Data
> *   Step 2: Implement Gaussian Naive Bayesian classifier
> *   Step 3: Build the classifier and check the correctness of Table building
> *   Step 4 Improve the classifier for Ranking
> *   Step 5: Make prediction and perform evaluation
> *   Step 6: Generate results

# **1. Basic Part (55%)**
In the first part, you need to implement the Naive Bayesian classifier:
- input features: ***9 physiological features***
- output prediction: ***hospital_death***

## *Import Packages*

> Note: You **cannot** import any other package

In [ ]:
import numpy as np
import pandas as pd
import csv
import math
import random
import pickle

## Global attributes
> You can add your own global attributes here


## Step 1: Load the input Data
Load the input file **hw2_basic_training.csv**

> Note: please don't change the input var name ***training_df, testing_df, X, and y***.

In [ ]:
# TODO: modify your file path
training_df = pd.read_csv('hw2_basic_training.csv')
testing_df = pd.read_csv('hw2_basic_testing.csv')
y = training_df['hospital_death']
X = training_df.drop('hospital_death', axis=1)

you can take a look at the input feature & ground truth format:

In [ ]:
X[:10]

In [ ]:
y[:10]

## Step 2: Implement Naive Bayesian classifier
In this part, you need to implement the Naive Bayesian classifier. Since the data is categorical, you can refer to the L3-Bayesian Classifier course slides, p.12~16. The Bayes' theorem is as follows:

$$P(C|X) = \frac{P(X|C) \cdot P(C)}{P(X)}$$

We know that taking the logarithm of a series of multiplications can be transformed into a series of additions, making it easier to calculate. So, we can formulate it as follows by taking the logarithm of both sides:

$$\log(P(C|X)) = \log(P(X|C)) + \log(P(C)) - \log(P(X))$$

The term $\log(P(X))$ is a normalization constant that ensures the probabilities sum to 1 across all classes and is the same for all classes. Since it's constant during prediction for all classes, it doesn't affect class comparison. Therefore, in practice, you don't need to compute or include $\log(P(X))$ explicitly when comparing classes. Instead, you can focus on the relative values of the posterior probabilities.

So the **final equation** for implementation will be:
$$\log(P(C|X)) = \log(P(X|C)) + \log(P(C))$$

However, this equation may still encounter issues if the likelihood $P(X|C)$ equals 0, leading to an undefined $\log(P(X|C))$. To handle this exception, a simple way to avoid the issue is to assume the addition of one new record to the table to calculate the likelihood:

_likelihood = 1 / (len(self.feature_probs_table[c][feature]) + 1)


In [ ]:
class NaiveBayes:
    def build_table(self, X, y):
        # classes for ground truth: there are only negative(0) and positive(1) for y (hospital_death)
        self.classes = np.unique(y)
        # record prior for two classes
        self.class_priors = {}
        # **feature_probs_table** is a 3D dictionary table:
        # structure: [class]    [feature]           [value] = probs
        # example:   [0]        ['gcs_eyes_apache'] [3]     = # of (hospital_death=0 && gcs_eyes_apache=3) / # of (hospital_death=0)
        # for more usage of python dict, you can refer to the link: https://www.w3schools.com/python/python_dictionaries.asp
        self.feature_probs_table = {}

        # consider negative(0) and positive(1) separately
        for c in self.classes:
            X_c = X[y == c]
            self.class_priors[c] = ... # TODO: calculate the prior
            self.feature_probs_table[c] = {}

            for feature in X.columns:
                self.feature_probs_table[c][feature] = {}
                for value in np.unique(X_c[feature]):
                    value_count = ... # TODO: Calculate the count of data points with the current feature value and the current class
                    total_count = ... # TODO: Calculate the total count of data points with the current feature within the current class
                    self.feature_probs_table[c][feature][value] = ... # TODO: Calculate and store the conditional probability of the feature value given the class

    def predict(self, X):
        predictions = [self._predict(x) for x in X.to_dict(orient='records')]
        return predictions

    def _predict(self, x):
        log_posteriors = []

        # this for loop implement: log(posteior) = log(prior) + log(likelihood)
        for c in self.classes:
            log_prior = np.log(self.class_priors[c])
            log_likelihood = 0
            for feature, value in x.items():
                if value in self.feature_probs_table[c][feature]:
                    # you can look up the table for the likelihood
                    _likelihood = ... # TODO: calculate likelihood by the self.feature_probs_table
                else:
                    # to fix issue of some cases not appear in the table
                    _likelihood = 1 / (len(self.feature_probs_table[c][feature]) + 1)
                log_likelihood += np.log(_likelihood)
            log_posterior = log_prior + log_likelihood
            log_posteriors.append((c, log_posterior))
        return ... # TODO: Return the class with the highest logarithm of posterior probability as the predicted class


## Step 3: Build the classifier and check the correctness of Table building
You can easily build an instance of your classifier and then create the table.

To check whether you have correctly built the table of the Naive Bayesian classifier, there is an example for you to ensure that your implementation is correct.


In [ ]:
# Create and build the dictionary table for Naive Bayes classifier
nb_classifier = NaiveBayes()
nb_classifier.build_table(X, y)

And you also need to output the dictionary variable ***feature_probs_table*** as a pickle file, which will be examined for correctness.
> Note: Since this is for checking the implementation of the build_table method, please ensure that the input for your table building, ***X and y,*** is taken from the provided hw2_basic_training.csv file ***BEFORE*** splitting the dataset into training and validation sets.

> Hint: Two values for you to check the implementation correctness:

> `nb_classifier.feature_probs_table[0]['gcs_eyes_apache'][3]` is
0.15299

> `nb_classifier.feature_probs_table[1]['gcs_eyes_apache'][3]` is
0.15896

In [ ]:
if round(nb_classifier.feature_probs_table[0]['gcs_eyes_apache'][3], 5) == 0.15299 and \
   round(nb_classifier.feature_probs_table[1]['gcs_eyes_apache'][3], 5) == 0.15896:
    print('pass')
else:
    print('fail')

pass


In [ ]:
# TODO: change your path to save the pickle file
pickle_file_path = 'hw2_basic_table'
with open(pickle_file_path, 'wb') as table_file:
    pickle.dump(nb_classifier.feature_probs_table, table_file)
    table_file.close()

## Step 4: Split Data
Split the data in *X, and y* into the training dataset and validation dataset.
> Note: You can use what you have implemented in hw1.

Since many students may not understand the meaning of a validation set, let's provide more explanation:

The purpose of a validation set is to determine whether our model is overfitting the training data.
- Underfitting: If the performance on the training set is poor (e.g., you haven't prepared enough for exam 1).
- Overfitting: If the performance on the training set is high, but the performance on the validation set is poor. (e.g., if you've focused solely on practicing with "past exam papers" (考古題) for exam 1, you might answer those questions correctly but struggle with new, unfamiliar questions during the actual exam).

If we achieve good performance on both the training set and the validation set, it indicates that the model has not only learned from the training data but also has the ability to make accurate inferences on unseen data.

![](https://hackmd.io/_uploads/SJLptEZWT.png)

Please split the dataset into training set and validation set

> Note: The purpose of ***random_state*** is to ensure that you can reproduce the results each time you split your dataset. This is often helpful for debugging.


In [ ]:
def train_val_split(X, y, val_size, random_state):
    ... # TODO: implement your own train_val_split

    return X_train, X_val, y_train, y_val

In [ ]:
# TODO: Split the data into training and validation sets
# Note: please follow template for the format of return variables
X_train, X_val, y_train, y_val = ... # TODO

## Step 5: Make predictions and perform evaluation
The method we will evaluate the performance of the Bayesian classifier is ***F1-socre***:

$$\text{precision} = \frac{\text{True Positives}}{\text{True Positives} + \text{False Positives}}$$

$$\text{Recall} = \frac{\text{True Positives}}{\text{True Positives} + \text{False Negatives}}$$

$$F1\text{-}score = \frac{2 \cdot \text{precision} \cdot \text{recall}}{\text{precision} + \text{recall}}$$

![](https://hackmd.io/_uploads/B1tfT9UWp.png)

Since the number of ground truth ***hospital_death*** where the outcome is positive is much less than the number of negative outcomes, we should focus on the f1-score of the positive class.

You need to implement the f1-score function to evaluate the performance of the Naive Bayesian classifier.

> Note: You should test your classifier by evaluating it on the training set and the validation set.


In [ ]:
def cal_f1_score(y_true, y_pred):
    # Calculate True Positives, False Positives, False Negatives
    tp = sum(1 for true, pred in zip(y_true, y_pred) if true == 1 and pred == 1)
    fp = ... # TODO: calculate the false positive
    fn = ... # TODO: calculate the false negative

    # Calculate precision and recall
    precision = ... # TDOO: calculate the precision
    recall = ... # TODO: calculate the recall

    # Calculate F1-score
    f1_score = ... # TODO: calculate the f1-score

    return f1_score

In [ ]:
... # TODO: build table on the training set
... # TODO: Make predictions on training set and calculate the f1-score
... # TODO: Make predictions on validation set and calculate the f1-score

## Step 6: Generate result
> Note: Please follow the format mension in the slides, the can only change the path for saving your code down below.

In [ ]:
predictions = ... # TODO: predict on the testing_df

# TODO: Specify the CSV file path
csv_file_path = 'hw2_basic_prediction.csv'

# Write the predictions to the CSV file
with open(csv_file_path, 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    # *** 10/21 update: header name ***
    writer.writerow(['hospital_death'])
    for prediction in predictions:
        writer.writerow([prediction])

# **1. Advanced Part (40%)**
In advanced part, you need to implement the Gaussian Bayesian classifier:
- input features: ***3 physiological features***
- output prediction: ***diabetes_mellitus***

## Global attributes
> You can add your own global attributes here


## Step 1: Load the input Data
Load the input file **hw2_advanced_training.csv**
> Note: please don't change the input var name ***training_df, testing_df, X, and y***.

In [ ]:
# TODO: modify your file path
training_df = pd.read_csv('hw2_advanced_training.csv')
testing_df = pd.read_csv('hw2_advanced_testing.csv')
y = training_df['diabetes_mellitus']
X = training_df.drop('diabetes_mellitus', axis=1)

In [ ]:
X[:10]

In [ ]:
y[:10]

you can check whether the standardization works

## Step 2: Implement Gaussian Naive Bayesian classifier
In this part, you need to implement the Gaussian Naive Bayesian classifier.

The main difference between Naive Bayesian and Gaussian Naive Bayesian is the likelihood part. For Gaussian NB, we can use the probability density function (PDF) of the ***Gaussian distribution*** (also known as the Normal distribution):

$$f(x) = \frac{1}{\sqrt{2\pi\sigma^2}} exp({-\frac{(x - \mu)^2}{2\sigma^2}})$$

The reason we need to use Gaussian is that when the data type is continuous numbers instead of discrete numbers, we can't build a table by just counting all the possible cases. However, we can assume the data distribution follows a Gaussian (or Normal) distribution by calculating its mean and variance. Then, we can approximate the values, even if some records don't appear in the training set.


In [ ]:
class GaussianNaiveBayesian:
    def build_table(self, X, y):
        # classes for ground truth: there are only negative(0) and positive(1) for y (hospital_death)
        self.classes = np.unique(y)
        # record prior for two classes
        self.class_priors = {}
        # **feature_mean_var_table** is a 3D dictionary table:
        # structure: [class]    [feature]           ['mean'] = mean
        # structure: [class]    [feature]           ['var']  = var
        # example:   [0]        ['gcs_eyes_apache'] ['mean'] = mean for feature='gcs_eyes_apache' when hospital_death=0
        # example:   [0]        ['gcs_eyes_apache'] ['var']  = var for feature='gcs_eyes_apache' when hospital_death=0
        self.feature_mean_var_table = {}
        for c in self.classes:
            X_c = X[y == c]
            self.class_priors[c] = ... # TODO: calculate prior
            self.feature_mean_var_table[c] = {}
            for feature in X.columns:
                self.feature_mean_var_table[c][feature] = {}
                # Calculate mean and var for each feature
                self.feature_mean_var_table[c][feature]['mean'] = ... # TDOO: calculate the mean
                # *** 10/19 note: make sure that if you call numpy.var, the ddof should set as 1 ***
                self.feature_mean_var_table[c][feature]['var'] = ...  # TODO: calculate the var

    def _calculate_likelihood(self, x, mean, var):
        return ... # TODO: calculate the Gaussian (normal) distribution pdf function as likelihoo

    def predict(self, X):
        predictions = [self._predict(x) for x in X.to_dict(orient='records')]
        return predictions

    def _predict(self, x):
        log_posteriors = []
        # this for loop implement: log(posteior) = log(prior) + log(likelihood)
        for c in self.classes:
            log_prior = np.log(self.class_priors[c])
            log_likelihood = 0
            for feature, value in x.items():
                log_likelihood += ... # TODO: calculate the log likelihood
            log_posterior = log_prior + log_likelihood
            log_posteriors.append((c, log_posterior))
        return ... # TODO: Return the class with the highest logarithm of posterior probability as the predicted class


## Step 3: Build the classifier and check the correctness of Table building
You can easily build an instance of your classifier and then create the table.

To check whether you have correctly built the table of the ***Gaussian Naive Bayesian classifier***, there is an example for you to ensure that your implementation is correct.


In [ ]:
# Initialize and build_table the model
gnb_classifier = GaussianNaiveBayesian()
gnb_classifier.build_table(X, y)

And you also need to output the dictionary variable ***feature_mean_var_table*** as a pickle file, which will be examined for correctness.
> Note: Since this is for checking the implementation of the build_table method, please ensure that the input for your table building, ***X and y,*** is taken from the provided hw2_advanced_training.csv file ***BEFORE*** splitting the dataset into training and validation sets.

> Hint: Two values for you to check the implementation correctness:


> `gnb_classifier.feature_mean_var_table[0]['bmi']['mean']` is
28.61544

> `gnb_classifier.feature_mean_var_table[0]['bmi']['var']` is
63.57263

In [ ]:
# *** 10/18 update: the value of mean and var***
if round(gnb_classifier.feature_mean_var_table[0]['bmi']['mean'], 5) == 28.61544 and \
   round(gnb_classifier.feature_mean_var_table[0]['bmi']['var'], 5) == 63.57263:
    print('pass')
else:
    print('fail')

pass


In [ ]:
# TODO: change your path to save the pickle file
pickle_file_path = 'hw2_advanced_table'
with open(pickle_file_path, 'wb') as table_file:
    pickle.dump(gnb_classifier.feature_mean_var_table, table_file)
    table_file.close()

## Step 4 Improve the classifier for Ranking 15%:

To make your model have better performance, you can try different ways to modify your model.

> hints (**you don't need to follow the hints**):

1. You can deal with the **outliers**
2. You can try first **converting real numbers into discrete values** and then using Naive Bayesian for classification.
3. You can try **def a new class for giving the prior a different weight** for decision-making.
4. Anything you want to try based on Bayesian.

> Note: You need to consider what kind of operations should also be performed on the testing_df.

In [ ]:
# TODO: you can try the hints written above to get higher ranking score

# training_df = pd.read_csv('hw2_advanced_training.csv') # TODO: modify your file path
# testing_df = pd.read_csv('hw2_advanced_testing.csv') # TODO: modify your file path
# Note: **you can change the order of following steps if you want**
# Note: **BUT, please make sure you have saved 'hw2_advanced_table' for submission BEFORE making the following improvements.**

# ... # hints(optional): deal with outliers
# ... # hints(optional): converting real numbers into discrete values
# ... # hints(optional): def a new class for giving the prior a different weight
# ... # hints: Split the data into training and validation sets
# ... # hints: build table for Bayesian Classifier
# ... # anything you want to try based on Bayesian

## Step 5: Make predictions and perform evaluation
You should test your model by evaluating the training set and validation set using the ***cal_f1_score*** function you implemented.


In [ ]:
... # TODO: build table on the training set
... # TODO: Make predictions on training set and calculate the f1-score
... # TODO: Make predictions on validation set and calculate the f1-score

## Step 6: Generate result
> Note: Please follow the format mentioned in the slides. You can only change the path for saving your code down below.


In [ ]:
predictions = ... # TODO: predict on the testing_df

# TODO: Specify the CSV file path
csv_file_path = 'hw2_advanced_prediction.csv'

# Write the predictions to the CSV file
with open(csv_file_path, 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    # *** 10/21 update: header name ***
    writer.writerow(['diabetes_mellitus'])
    for prediction in predictions:
        writer.writerow([prediction])

# Report *(5%)*

Report should be submitted as a pdf file **hw2_report.pdf**

* Briefly describe why we take log when implement the Bayesian classifier? (1%)
* Briefly describe the difference between Naïve Bayesian and Gaussian Naïve Bayesian classifier? (1%)
* Briefly describe the difficulty you encountered (1%)
* Summarize how you solve the difficulty and your reflections (2%)
* **No more than one page**

# Save the Code File
Please save your code and submit it as an ipynb file! (**hw2.ipynb**)

# Submission:
1. hw2_basic_prediction.csv
2. hw2_basic_table: **make sure you build_table BEFORE split train_val set, and pass the given example**
3. hw2_advanced_prediction.csv
4. hw2_advanced_table: **make sure you build_table BEFORE split train_val set or pre-processing, and pass the given example**
5. hw2.ipynb
6. hw2_report.pdf